In [ ]:
#pip install djitellopy opencv-python numpy ultralytics

# Velocity based takeoff

In [ ]:
from djitellopy import Tello
import cv2
import numpy as np
import time
import logging
import os
from ultralytics import YOLO

# Suppress INFO and WARNING messages from djitellopy library
logging.getLogger('djitellopy').setLevel(logging.ERROR)

#####################################
# Load YOLO model for adversary detection
#####################################
model_path = "/Users/hridayunadkat/Desktop/ECE 532/Final Project/Final Project-Image Classification/doguilmak image classification file/best.pt"
if not os.path.exists(model_path):
    raise FileNotFoundError(f"YOLO model not found at {model_path}")

# Load the YOLO model
model = YOLO(model_path)
print(f"YOLO model loaded from {model_path}")

# Threshold for bounding box area to consider as adversary
ADVERSARY_BBOX_AREA_THRESHOLD = 10000  # pixels^2

'''
World building
'''
#####################################
# Initialization of coordinates
#####################################
# Coordinate system:
#   x: positive = forward, negative = backward
#   y: positive = left, negative = right
#   z: positive = up, negative = down
# Initial starting position of ego drone (at the center of the picnic blanket)
ego_x = 0
ego_y = 0
ego_z = 0

# State history: stores full state (x, y, z) at each time step
state_history = []  # List of tuples: [(x, y, z), ...]

# Flags to track alternative paths (reset at start of each iteration)
alternative_path_x_taken = False  # Prohibits x-direction movement temporarily
alternative_path_y_taken = False  # Prohibits y-direction movement temporarily

#####################################
# Adversary detection settings
#####################################
adversary_active = True  # Set to True to enable adversary avoidance (using YOLO detection)

# Estimated adversary position (updated based on bounding box detections)
adversary_x_estimate = None  # Estimated x position in world coordinates
adversary_y_estimate = None  # Estimated y position in world coordinates
adversary_position_initialized = False  # True after first 5-second detection period

# Threshold distance for collision avoidance (squared distance: x^2 + y^2)
avoidance_radius = 50  # cm threshold
adversary_threshold = avoidance_radius**2  # 30^2 = 900 (30 cm threshold)

# Bounding box area to distance mapping (pixels² -> cm)
# Calibration points:
# - 10,000-20,000 pixels² → 80 cm
# - 70,000-120,000 pixels² → 40 cm
# - > 300,000 pixels² → 5 cm (dangerous)
def estimate_distance_from_bbox_area(area):
    # Define range endpoints and corresponding distances
    area_very_far = 10000
    area_far_end = 20000  # Also area_mid_start
    distance_far = 80  # cm at area_far_end
    
    area_mid_end = 50000  # Also area_close_start
    distance_mid = 40  # cm at area_mid_end
    
    area_close_end = 100000  # Also area_danger_start
    
    area_danger_end = 300000
    distance_danger = 5  # cm at area_danger_end
    
    if area < area_very_far:
        return 100  # Very far, assume 100cm
    elif area_very_far <= area <= area_far_end:
        return distance_far  # Calibrated: 10k-20k = 80cm
    elif area_far_end < area < area_mid_end:
        # Interpolate between distance_far (at area_far_end) and distance_mid (at area_mid_end)
        # slope = (distance_mid - distance_far) / (area_mid_end - area_far_end)
        # distance = distance_far + slope * (area - area_far_end)
        slope = (distance_mid - distance_far) / (area_mid_end - area_far_end)
        distance = distance_far + slope * (area - area_far_end)
        return max(distance_mid, min(distance_far, distance))  # Clamp between distance_mid and distance_far
    elif area_mid_end <= area <= area_close_end:
        return distance_mid  # Calibrated: 70k-120k = 40cm
    elif area_close_end < area < area_danger_end:
        # Interpolate between distance_mid (at area_close_end) and distance_danger (at area_danger_end)
        # slope = (distance_danger - distance_mid) / (area_danger_end - area_close_end)
        # distance = distance_mid + slope * (area - area_close_end)
        slope = (distance_danger - distance_mid) / (area_danger_end - area_close_end)
        distance = distance_mid + slope * (area - area_close_end)
        return max(distance_danger, min(distance_mid, distance))  # Clamp between distance_danger and distance_mid
    else:  # area >= area_danger_end
        return distance_danger  # Dangerous: >=300k = 5cm

#####################################
# Goal tracking
#####################################
goal_index = 0
goals = [
    # (20, 0),   # Goal 1: Forward
    # (20, 20),  # Goal 2: Forward and left
    # (20, -20)   # Goal 3: Forward and Right Two Steps

    # (0, 20),   # Goal 1: Left
    # (20, 20),  # Goal 2: Left, then move Forward
    # (20, -20)   # Goal 3: Left, Forward, then Right Two Steps

    (0, 80),   # Goal 1: Left
    (40, 80),  # Goal 2: Left, then move Forward
    (40, -80)   # Goal 3: Left, Forward, then Right Two Steps
]

'''
Drone building
'''
#####################################
# Initialization of motion params
#####################################
upward_initialization =50 #20 cm is the minimum
move_step = 40

#####################################
# Initialization of drone
#####################################
tello = Tello()
#print("Connecting to Tello...")
tello.connect()
print(f"Battery level: {tello.get_battery()}%")
tello.streamon()

print("Press 't' to takeoff, 'q' to land and quit.")
drone_in_air = False

#####################################
# Definitions
#####################################
def estimate_adversary_position_from_bbox(frame, box, current_ego_x, current_ego_y, is_initial_period=False):
    global adversary_x_estimate, adversary_y_estimate
    
    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
    box_area = (x2 - x1) * (y2 - y1)
    
    # Estimate distance (depth) from bounding box area
    distance_cm = estimate_distance_from_bbox_area(box_area)
    
    # Adversary is assumed to be directly ahead (y = 0 relative to drone) and stationary
    # Depth (x) calculation:
    # - During initial period: calculate from bounding box area
    # - After initial period: keep the same depth (adversary is stationary)
    if is_initial_period or adversary_x_estimate is None:
        # Calculate depth: adversary is ahead by distance_cm
        adversary_x_estimate = current_ego_x + distance_cm
        # Adversary y is same as drone y (directly ahead, y=0 relative)
        adversary_y_estimate = current_ego_y
        print(f"  Calculated adversary depth: {distance_cm:.1f} cm ahead (from bbox area {box_area:.0f})")
    else:
        # After initial period: adversary is stationary
        # Keep the same x (depth) - don't recalculate
        # Keep y = 0 (same as initial y, adversary doesn't move)
        # adversary_x_estimate and adversary_y_estimate stay constant
        pass
    
    return adversary_x_estimate, adversary_y_estimate

def detect_adversary_yolo(frame, current_ego_x=None, current_ego_y=None, update_position=False, is_initial_period=False):
    if frame is None:
        return False, None, 0
    
    # Run YOLO detection
    results = model(frame)
    
    # Check for bounding boxes with area > threshold
    boxes = results[0].boxes
    adversary_detected = False
    max_area = 0
    largest_box = None
    
    if len(boxes) > 0:
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            box_area = (x2 - x1) * (y2 - y1)
            if box_area > max_area:
                max_area = box_area
                largest_box = box
            if box_area > ADVERSARY_BBOX_AREA_THRESHOLD:
                adversary_detected = True
    
    # Update adversary position estimate if requested and we have a detection
    if update_position and largest_box is not None and current_ego_x is not None and current_ego_y is not None:
        adv_x, adv_y = estimate_adversary_position_from_bbox(frame, largest_box, current_ego_x, current_ego_y, is_initial_period=is_initial_period)
        # Print position estimate for each detection
        print(f"Adversary position estimate: ({adv_x:.1f}, {adv_y:.1f}) [Drone at: ({current_ego_x}, {current_ego_y})]")
    
    # Get annotated frame (always plot to show detections, even if below threshold)
    annotated_frame = results[0].plot()
    
    return adversary_detected, annotated_frame, max_area

def compute_switching_filter(new_x, new_y, current_frame=None, current_ego_x=None, current_ego_y=None):
    if not adversary_active:
        return False
    
    # If we don't have an estimated adversary position yet, movement is safe
    if adversary_x_estimate is None or adversary_y_estimate is None:
        return False
    
    # Calculate squared distance from intended new position to estimated adversary position
    dist_to_adversary_sq = (new_x - adversary_x_estimate)**2 + (new_y - adversary_y_estimate)**2
    
    # Block if within avoidance radius (30 cm)
    is_blocking = dist_to_adversary_sq < adversary_threshold
    
    if is_blocking:
        distance = np.sqrt(dist_to_adversary_sq)
        print(f"Movement to ({new_x}, {new_y}) BLOCKED: too close to adversary at ({adversary_x_estimate:.1f}, {adversary_y_estimate:.1f}), distance = {distance:.1f} cm < {avoidance_radius} cm")
    else:
        distance = np.sqrt(dist_to_adversary_sq)
        print(f"Movement to ({new_x}, {new_y}) SAFE: distance to adversary at ({adversary_x_estimate:.1f}, {adversary_y_estimate:.1f}) = {distance:.1f} cm >= {avoidance_radius} cm")
    
    return is_blocking

def find_alternative_path(current_x, current_y, goal_x, goal_y, blocked_direction='x', current_frame=None):
    dx = goal_x - current_x  # positive = need to go forward, negative = need to go backward
    dy = goal_y - current_y  # positive = need to go left, negative = need to go right
    
    alternatives = []
    
    # If x-direction is blocked, try moving in y-direction
    if blocked_direction == 'x':
        # First try moving in the y-direction that helps reach the goal (if dy != 0)
        if dy != 0:
            # Try moving in the direction of the goal's y component
            alt_x = current_x
            alt_y = current_y + move_step if dy > 0 else current_y - move_step
            if not compute_switching_filter(alt_x, alt_y, current_frame):
                alternatives.append((alt_x, alt_y))
        
        # If that doesn't work, try the opposite y direction
        if not alternatives and dy != 0:
            alt_x = current_x
            alt_y = current_y - move_step if dy > 0 else current_y + move_step
            if not compute_switching_filter(alt_x, alt_y, current_frame):
                alternatives.append((alt_x, alt_y))
        
        # If that doesn't work or dy == 0, try left and right (to go around obstacle)
        if not alternatives:
            # Try left first (positive y)
            alt_x = current_x
            alt_y = current_y + move_step
            if not compute_switching_filter(alt_x, alt_y, current_frame):
                alternatives.append((alt_x, alt_y))
        
        # If left doesn't work, try right
        if not alternatives:
            alt_x = current_x
            alt_y = current_y - move_step
            if not compute_switching_filter(alt_x, alt_y, current_frame):
                alternatives.append((alt_x, alt_y))
    
    # If y-direction is blocked, try moving in x-direction
    elif blocked_direction == 'y':
        # First try moving in the x-direction that helps reach the goal (if dx != 0)
        if dx != 0:
            # Try moving in the direction of the goal's x component
            alt_x = current_x + move_step if dx > 0 else current_x - move_step
            alt_y = current_y
            if not compute_switching_filter(alt_x, alt_y, current_frame):
                alternatives.append((alt_x, alt_y))
        
        # If that doesn't work or dx == 0, try forward and backward (to go around obstacle)
        if not alternatives:
            # Try forward first (positive x)
            alt_x = current_x + move_step
            alt_y = current_y
            if not compute_switching_filter(alt_x, alt_y, current_frame):
                alternatives.append((alt_x, alt_y))
        
        # If forward doesn't work, try backward
        if not alternatives:
            alt_x = current_x - move_step
            alt_y = current_y
            if not compute_switching_filter(alt_x, alt_y, current_frame):
                alternatives.append((alt_x, alt_y))
    
    # Return the first safe alternative found (prioritized by goal direction)
    if alternatives:
        return alternatives[0]
    
    return None  # No safe alternative path found






'''
Takeoff
'''
#####################################
# Drone maneuverance after takeoff
#####################################
try:
    # Record initial state before loop starts
    state_history.append((ego_x, ego_y, ego_z))
    
    while True:
        frame = tello.get_frame_read().frame
        if frame is None:
            continue
        frame = cv2.resize(frame, (720, 480))
        h, w, _ = frame.shape

        # Only run YOLO detection when drone is in the air (to avoid blocking key presses before takeoff)
        if drone_in_air and adversary_active:
            # Run YOLO detection on frame (update position estimate during flight)
            # After initial period, depth is kept constant (adversary is stationary)
            adversary_detected, annotated_frame, max_bbox_area = detect_adversary_yolo(
                frame, 
                current_ego_x=ego_x, 
                current_ego_y=ego_y, 
                update_position=True,
                is_initial_period=False  # After initialization, depth stays constant
            )
            
            # Display the annotated frame with YOLO detections
            display_frame = annotated_frame if annotated_frame is not None else frame.copy()
            
            # Add text overlay showing detection status
            if adversary_detected:
                cv2.putText(display_frame, f"ADVERSARY DETECTED! Area: {max_bbox_area:.0f}", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                if adversary_x_estimate is not None and adversary_y_estimate is not None:
                    cv2.putText(display_frame, f"Est. pos: ({adversary_x_estimate:.0f}, {adversary_y_estimate:.0f})", 
                               (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            else:
                if max_bbox_area > 0:
                    cv2.putText(display_frame, f"Detection below threshold (area: {max_bbox_area:.0f})", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                else:
                    cv2.putText(display_frame, f"No detections", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            # Before takeoff, just show raw frame (no YOLO detection to keep loop fast)
            display_frame = frame.copy()
            cv2.putText(display_frame, "Press 't' to takeoff", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.imshow("Tello Tracking Feed", display_frame)

        key = cv2.waitKey(1) & 0xFF

        # Controls for drone
        if key == ord('t') and not drone_in_air:
            tello.takeoff()
            drone_in_air = True
            print("Drone is now airborne and hovering.")
            tello.move_up(upward_initialization)  # move up 30 cm after takeoff
            ego_z += upward_initialization
            state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
            print(f"Drone moved up {upward_initialization} cm. Current state: ({ego_x}, {ego_y}, {ego_z})")
            
            # Wait a moment for drone to stabilize after upward movement
            time.sleep(1.0)
            print("Drone stabilized. Starting adversary detection initialization...")
            
            if adversary_active:
                print(f"YOLO adversary detection active. Bounding box area threshold: {ADVERSARY_BBOX_AREA_THRESHOLD} pixels^2")
                print("Hovering for 5 seconds to initialize adversary position detection...")
                
                # Hover for 5 seconds and perform detections to estimate adversary position
                detection_start_time = time.time()
                detection_count = 0
                while time.time() - detection_start_time < 5.0:
                    # Get frame for detection
                    detection_frame = tello.get_frame_read().frame
                    if detection_frame is not None:
                        detection_frame = cv2.resize(detection_frame, (720, 480))
                        # Run detection with position update enabled (is_initial_period=True)
                        adv_detected, annotated_frame, max_area = detect_adversary_yolo(
                            detection_frame, 
                            current_ego_x=ego_x, 
                            current_ego_y=ego_y, 
                            update_position=True,
                            is_initial_period=True
                        )
                        detection_count += 1
                        
                        # Display the annotated frame with detection info
                        display_frame = annotated_frame if annotated_frame is not None else detection_frame.copy()
                        elapsed = time.time() - detection_start_time
                        cv2.putText(display_frame, f"Initial Detection Period: {elapsed:.1f}s / 5.0s", 
                                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                        if adv_detected:
                            cv2.putText(display_frame, f"ADVERSARY DETECTED! Area: {max_area:.0f}", 
                                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                            if adversary_x_estimate is not None and adversary_y_estimate is not None:
                                cv2.putText(display_frame, f"Est. pos: ({adversary_x_estimate:.0f}, {adversary_y_estimate:.0f})", 
                                           (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                        cv2.imshow("Tello Tracking Feed", display_frame)
                        cv2.waitKey(1)  # Update display
                        
                        if adv_detected:
                            print(f"  Detection {detection_count}: Adversary detected! Area: {max_area:.0f} pixels²")
                    
                    time.sleep(0.1)  # Small delay between detections
                
                adversary_position_initialized = True
                print(f"Initial detection period complete ({detection_count} detections).")
                if adversary_x_estimate is not None and adversary_y_estimate is not None:
                    print(f"Initial adversary position estimate: ({adversary_x_estimate:.1f}, {adversary_y_estimate:.1f})")
                else:
                    print("No adversary detected during initialization period.")

        # if key == ord('t') and not drone_in_air:
        #     tello.takeoff()
        #     time.sleep(2)  # Delay after takeoff to allow drone to stabilize
        #     drone_in_air = True
        #     print("Drone is now airborne.")
            
        #     # Gradual takeoff using velocity control
        #     target_height = 30  # cm you want to reach (minimum safe height)
        #     hover_speed = 10    # cm/s (upward velocity)
        #     steps = int(target_height / hover_speed)
            
        #     for _ in range(steps):
        #         tello.send_rc_control(0, 0, hover_speed, 0)  # (left-right, forward-back, up-down, yaw)
        #         ego_z += hover_speed
        #         state_history.append((ego_x, ego_y, ego_z))
        #         time.sleep(1)  # sleep 1 second per step

        #     # Stop upward motion and hover
        #     #tello.send_rc_control(0, 0, 0, 0)
        #     time.sleep(0.5)  # Delay after stopping motion
        #     print(f"Reached hover height: {ego_z} cm")
        #     if adversary_active:
        #         print(f"YOLO adversary detection active. Bounding box area threshold: {ADVERSARY_BBOX_AREA_THRESHOLD} pixels^2")

        


        # drone movement based on goals
        if drone_in_air and goal_index < len(goals):
            target_x, target_y = goals[goal_index]
            
            # Check if goal is blocked by adversary (switching-based control)
            if adversary_active and compute_switching_filter(target_x, target_y, frame):
                print(f"Goal {goal_index + 1} at ({target_x}, {target_y}) is BLOCKED by adversary (YOLO detection)")
                print(f"Skipping goal {goal_index + 1} and moving to next goal...")
                goal_index += 1
                time.sleep(0.5)
                
                # Check if we've processed all goals (no linked list - don't return to skipped goals)
                if goal_index >= len(goals):
                    print("All goals processed! Landing the drone...")
                    if drone_in_air:
                        tello.land()
                        time.sleep(3)  # Delay after landing
                        state_history.append((ego_x, ego_y, ego_z))  # Record final state
                        print("Drone has landed.")
                        print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                        print(f"Total states recorded: {len(state_history)}")
                    break
                continue
            
            # Calculate distance to current goal
            dx = target_x - ego_x
            dy = target_y - ego_y
            distance_to_goal = np.sqrt(dx**2 + dy**2)
            
            # Threshold for considering goal reached (within 5 cm)
            goal_threshold = 5
            
            if distance_to_goal > goal_threshold:
                movement_made = False

                # Initialize flags for this movement cycle (reset at end of each iteration)
                #alternative_path_x_taken = False
                #alternative_path_y_taken = False
                print(f"Starting next movement cycle, for now alternative_path_x_taken is {alternative_path_x_taken} and alternative_path_y_taken is {alternative_path_y_taken}")

                # Move in x direction first (only if not prohibited by alternative path)
                if abs(dx) > 0 and not alternative_path_x_taken:
                    # Calculate potential new position
                    new_x = ego_x + move_step if dx > 0 else ego_x - move_step
                    new_y = ego_y  # y stays the same for x movement
                    
                    # Check if movement is safe (if adversary is active)
                    can_move_x = True
                    if adversary_active:
                        if compute_switching_filter(new_x, new_y, frame):
                            can_move_x = False
                            print(f"CANT MOVE TOWARDS GOAL {goal_index + 1} at {goals[goal_index]} IN X DIRECTION, ADVERSARY BLOCKING (YOLO)")
                            # Try alternative path (x-direction is blocked, so try y-direction)
                            alt_waypoint = find_alternative_path(ego_x, ego_y, target_x, target_y, blocked_direction='x', current_frame=frame)
                            if alt_waypoint:
                                alt_x, alt_y = alt_waypoint
                                alt_dx = alt_x - ego_x
                                alt_dy = alt_y - ego_y
                                # Print and execute alternative movement
                                if abs(alt_dy) > 0:
                                    # Alternative path moves in y direction
                                    if alt_dy > 0:
                                        print(f"Taking alternative path: Moving left to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_left(move_step)  # positive y = left
                                        time.sleep(0.5)  # Delay after movement
                                        ego_y += move_step
                                    else:
                                        print(f"Taking alternative path: Moving right to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_right(move_step)  # negative y = right
                                        time.sleep(0.5)  # Delay after movement
                                        ego_y -= move_step
                                    alternative_path_y_taken = True  # Prohibit y-direction temporarily
                                elif abs(alt_dx) > 0:
                                    # Alternative path moves in x direction
                                    if alt_dx > 0:
                                        print(f"Taking alternative path: Moving forward to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_forward(move_step)  # positive x = forward
                                        time.sleep(0.5)  # Delay after movement
                                        ego_x += move_step
                                    else:
                                        print(f"Taking alternative path: Moving back to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_back(move_step)  # negative x = backward
                                        time.sleep(0.5)  # Delay after movement
                                        ego_x -= move_step
                                    alternative_path_x_taken = True  # Prohibit x-direction temporarily
                                state_history.append((ego_x, ego_y, ego_z))
                                print(f"Alternative path taken. Current position: ({ego_x:.1f}, {ego_y:.1f}); alternative_path_x_taken is {alternative_path_x_taken}, alternative_path_y_taken is {alternative_path_y_taken} ")
                                movement_made = True
                                time.sleep(0.5)
                            else:
                                print(f"No alternative path found for X movement. Trying Y direction or skipping...")
                    
                    # Execute movement if safe
                    if can_move_x and not alternative_path_x_taken:
                        if dx > 0:
                            tello.move_forward(move_step)  # positive x = forward
                            time.sleep(0.5)  # Delay after movement
                        else:
                            tello.move_back(move_step)  # negative x = backward
                            time.sleep(0.5)  # Delay after movement
                        ego_x = new_x
                        state_history.append((ego_x, ego_y, ego_z))
                        print(f"Can move x. Moving towards goal {goal_index + 1}: ({target_x}, {target_y}), Current: ({ego_x:.1f}, {ego_y:.1f})")
                        movement_made = True
                        time.sleep(0.5)

                # Then move in y direction (only if not prohibited and x movement didn't happen or was successful)
                if abs(dy) > 0 and not alternative_path_y_taken and (not movement_made or abs(dx) == 0 or alternative_path_x_taken):
                    # Calculate potential new position
                    new_x = ego_x  # x stays the same for y movement
                    new_y = ego_y + move_step if dy > 0 else ego_y - move_step
                    
                    # Check if movement is safe (if adversary is active)
                    can_move_y = True
                    if adversary_active:
                        if compute_switching_filter(new_x, new_y, frame):
                            can_move_y = False
                            print(f"CANT MOVE TOWARDS GOAL {goal_index + 1} at {goals[goal_index]} IN Y DIRECTION, ADVERSARY BLOCKING (YOLO)")
                            # Try alternative path (y-direction is blocked, so try x-direction)
                            alt_waypoint = find_alternative_path(ego_x, ego_y, target_x, target_y, blocked_direction='y', current_frame=frame)
                            if alt_waypoint:
                                alt_x, alt_y = alt_waypoint
                                alt_dx = alt_x - ego_x
                                alt_dy = alt_y - ego_y
                                # Print and execute alternative movement
                                if abs(alt_dx) > 0:
                                    # Alternative path moves in x direction
                                    if alt_dx > 0:
                                        print(f"Taking alternative path: Moving forward to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_forward(move_step)  # positive x = forward
                                        time.sleep(0.5)  # Delay after movement
                                        ego_x += move_step
                                    else:
                                        print(f"Taking alternative path: Moving back to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_back(move_step)  # negative x = backward
                                        time.sleep(0.5)  # Delay after movement
                                        ego_x -= move_step
                                    alternative_path_x_taken = True  # Prohibit x-direction temporarily
                                elif abs(alt_dy) > 0:
                                    # Alternative path moves in y direction
                                    if alt_dy > 0:
                                        print(f"Taking alternative path: Moving left to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_left(move_step)  # positive y = left
                                        time.sleep(0.5)  # Delay after movement
                                        ego_y += move_stepz
                                    else:
                                        print(f"Taking alternative path: Moving right to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_right(move_step)  # negative y = right
                                        time.sleep(0.5)  # Delay after movement
                                        ego_y -= move_step
                                    alternative_path_y_taken = True  # Prohibit y-direction temporarily
                                state_history.append((ego_x, ego_y, ego_z))
                                print(f"Alternative path taken. Current position: ({ego_x:.1f}, {ego_y:.1f}); alternative_path_x_taken is {alternative_path_x_taken}, alternative_path_y_taken is {alternative_path_y_taken}")
                                movement_made = True
                                time.sleep(0.5)
                            else:
                                print(f"No alternative path found for Y movement.")
                    
                    # Execute movement if safe
                    if can_move_y and not alternative_path_y_taken:
                        if dy > 0:
                            tello.move_left(move_step)  # positive y = left
                            time.sleep(0.5)  # Delay after movement
                        else:
                            tello.move_right(move_step)  # negative y = right
                            time.sleep(0.5)  # Delay after movement
                        ego_y = new_y
                        state_history.append((ego_x, ego_y, ego_z))
                        print(f"Can move y. Moving towards goal {goal_index + 1}: ({target_x}, {target_y}), Current: ({ego_x:.1f}, {ego_y:.1f})")
                        movement_made = True
                        time.sleep(0.5)

                #Strategic reset of flags
                if alternative_path_x_taken:
                    if ego_y == target_y:
                        alternative_path_x_taken = False
                        print("Resetting alternative_path_x_taken to False, since we are at the target y coordinate")

                if alternative_path_y_taken:
                    if ego_x == target_x:
                        alternative_path_y_taken = False
                        print("Resetting alternative_path_y_taken to False, since we are at the target x coordinate")


                # If completely stuck (can't move in either direction and no alternative path)
                if not movement_made and adversary_active:
                    print(f"STUCK: Cannot move towards goal {goal_index + 1} from current position ({ego_x}, {ego_y})")
                    print(f"Skipping goal {goal_index + 1} and moving to next goal...")
                    goal_index += 1
                    time.sleep(0.5)
                    
                    # Check if we've processed all goals
                    if goal_index >= len(goals):
                        print("All goals processed! Landing the drone...")
                        if drone_in_air:
                            tello.land()
                            time.sleep(3)  # Delay after landing
                            state_history.append((ego_x, ego_y, ego_z))  # Record final state
                            print("Drone has landed.")
                            print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                            print(f"Total states recorded: {len(state_history)}")
                        break

            
            else:
                # Goal reached
                print(f"Goal {goal_index + 1} reached! ({target_x}, {target_y})")
                goal_index += 1
                time.sleep(1)
                
                if goal_index >= len(goals):
                    print("All goals processed! Landing the drone...")
                    if drone_in_air:
                        tello.land()
                        time.sleep(3)  # Delay after landing
                        state_history.append((ego_x, ego_y, ego_z))  # Record final state
                        print("Drone has landed.")
                        print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                        print(f"Total states recorded: {len(state_history)}")
                    break




        # Landing control
        if key == ord('q'):
            if drone_in_air:
                tello.land()
                time.sleep(3)  # Delay after landing to allow drone to settle
                state_history.append((ego_x, ego_y, ego_z))  # Record final state before landing
                print("Drone has landed.")
                print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                print(f"Total states recorded: {len(state_history)}")
            break



#####################################
# Landing the drone / finish program
#####################################
except Exception as e:
    print(f"Error occurred: {e}")
    if drone_in_air:
        print("Error occurred. Landing the drone.")
        tello.land()
        time.sleep(3)  # Delay after landing

except KeyboardInterrupt:
    print("Keyboard Interrupt received. Landing the drone.")
    if drone_in_air:
        tello.land()
        time.sleep(3)  # Delay after landing
    tello.streamoff()
    cv2.destroyAllWindows()

finally:
    tello.streamoff()
    cv2.destroyAllWindows()


YOLO model loaded from /Users/hridayunadkat/Desktop/ECE 532/Final Project/Final Project-Image Classification/doguilmak image classification file/best.pt
Battery level: 86%
Press 't' to takeoff, 'q' to land and quit.
Drone is now airborne and hovering.
Drone moved up 50 cm. Current state: (0, 0, 50)
Drone stabilized. Starting adversary detection initialization...
YOLO adversary detection active. Bounding box area threshold: 10000 pixels^2
Hovering for 5 seconds to initialize adversary position detection...

0: 448x640 1 drone, 2017.3ms
Speed: 3.0ms preprocess, 2017.3ms inference, 2.5ms postprocess per image at shape (1, 3, 448, 640)
  Calculated adversary depth: 80.0 cm ahead (from bbox area 10168)
Adversary position estimate: (80.0, 0.0) [Drone at: (0, 0)]
  Detection 1: Adversary detected! Area: 10168 pixels²

0: 448x640 1 drone, 1138.8ms
Speed: 3.9ms preprocess, 1138.8ms inference, 2.2ms postprocess per image at shape (1, 3, 448, 640)
  Calculated adversary depth: 100.0 cm ahead (fro

TelloException: Command 'land' was unsuccessful for 4 tries. Latest response:	'error'